<a href="https://colab.research.google.com/github/Shin-mat/Google-Colab/blob/main/distil_whisper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install --upgrade pip
!pip install --upgrade git+https://github.com/huggingface/transformers.git accelerate datasets[audio]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-4tuu89sq
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-4tuu89sq
  Resolved https://github.com/huggingface/transformers.git to commit 3cefac1d974db5e2825a0cb2b842883a628be7a0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.0 MB/s eta

In [5]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

# Add new tokens to the tokenizer
new_tokens = ["new_token"]  # replace with your new tokens
added_tokens = processor.tokenizer.add_tokens(new_tokens)

# Resize the token embeddings of the model
model.resize_token_embeddings(len(processor.tokenizer))

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=0 if torch.cuda.is_available() else -1
)

import subprocess

# Path of input file
input_file = "A-F_conv.m4a"
# Path of output file
output_file = "A-F_conv.wav"

# Converting audio files with ffmpeg
subprocess.run(["ffmpeg", "-i", input_file, output_file])

result = pipe(output_file)  # Corrected this line
print(result["text"])
with open('output.txt', 'w', encoding='utf-8') as f:
    f.write(result["text"])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.


 I also replied I was so angry yeah and then like what kind of thing is that urgent that you need to email us twice all the time in the evening I messaged you in the morning you are busy yourself and then when you get time at 10 30 11 o'clock at night you start messaging and you expect answers yeah just like i i i talked the same thing to pervane it's like he paid someone for eight hours but then he expected more 14 hours it's not also about hours for now some work some things are more you know you can do your experiment for eight hours you might not be so tired but you know when you're doing these kind of things it's like not about hours it's more like how much your mind is getting exhausted yes right this kind of work for i think two hours also is very taxing yes and also like at least show some appreciation like no you just say one thank you and you think you're done with it everything I'm dead I know it's so annoying I totally no no no you're on 20th like two days are left today is